# Lab 9
Understanding Docker Networks

What is Container Networking?
Container networking also called Software Defined Networking (SDN) is what allows Docker networkgin subsystems to work. 

Some defalt Docker Netwrok drivers are:
- Bridge
    - a Network that is User-defined to allow for all containers connected on the same network to communicate with eachother. 
- Host
    - a container is attahed to a Docker host. Any traffic routed to the container must be routed from the host system.
- Overlay
    - Created in a network spanning multiple docker hosts. Think each of your docker containers in the lab talking to eachother
- Macvlan
    - used in complicated networking to assing logical addresses based on MAC addresses to the virtual network interfaces making it apear as if the container has a physical network interaface connected to the network. Used in legacy applications.
- None
    -  the container only has a loopback interface so it can talk to itself.



# Part 1 - networking

In [1]:
!docker run -d --name nginx2 --network=none -p 81:81 nginx

3985c4706df2fed6d1aa01bd6a68cfb4da0e08896cf92019a8aa360bf2d8b845


we now have an nginx container 

In [2]:
!curl localhost

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
curl: (7) Failed to connect to localhost port 80 after 2258 ms: Could not connect to server


In [3]:
!docker exec -it nginx /bin/sh -c "curl google.com"



Error response from daemon: No such container: nginx


As you can see the nginx container can not talk to google.com

Docker comes with a subcommand for handling Docker networks. To get started, try the following command:

In [4]:
!docker network

Usage:  docker network COMMAND

Manage networks

Commands:
  connect     Connect a container to a network
  create      Create a network
  disconnect  Disconnect a container from a network
  inspect     Display detailed information on one or more networks
  ls          List networks
  prune       Remove all unused networks
  rm          Remove one or more networks

Run 'docker network COMMAND --help' for more information on a command.


In [5]:
!docker network ls # see what networks are avalable

docker: 'docker network ls' accepts no arguments

Usage:  docker network ls [OPTIONS]

Run 'docker network ls --help' for more information


In [6]:
!docker network inspect none # note the nginx

Error response from daemon: network # not found
Error response from daemon: network note not found
Error response from daemon: network the not found
Error response from daemon: network nginx not found


[
    {
        "Name": "none",
        "Id": "ef5b222716f397a8f469d0c7443ca384ed14e51768fc3ec44a2893e30356df66",
        "Created": "2025-04-07T01:48:55.336663273Z",
        "Scope": "local",
        "Driver": "null",
        "EnableIPv4": true,
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": null,
            "Config": null
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {
            "3985c4706df2fed6d1aa01bd6a68cfb4da0e08896cf92019a8aa360bf2d8b845": {
                "Name": "nginx2",
                "EndpointID": "eb88e5dc1622aca1c7d239b263ce789b67a90c8b87103f015629a80dc92f59dc",
                "MacAddress": "",
                "IPv4Address": "",
                "IPv6Address": ""
            }
        },
        "Options": {},
        "Labels": {}
    }
]


In [7]:
#setup another conainer for mysql
!docker run -d --name mysql -p 3306:3306 -e MYSQL_ROOT_PASSWORD=1 mysql:8
#Note the -d means detached mode so no container is using created just the MySQL database 
!docker logs -f mysql

docker: Error response from daemon: Conflict. The container name "/mysql" is already in use by container "443f7ff4af56184926c4637e0d976a6b7026bf52a7478ba8ad9afac6f214ecc6". You have to remove (or rename) that container to be able to reuse that name.

Run 'docker run --help' for more information


^C


## The following needs to be run in the terminal

In [ ]:
!docker run -d --name adminer -p 8080:8080 adminer


docker: Error response from daemon: Conflict. The container name "/adminer" is already in use by container "11eaeb5a4eb5b303320bf7e5964ffb5b07f82df0ff8190da993bf36e19fcc063". You have to remove (or rename) that container to be able to reuse that name.

Run 'docker run --help' for more information


In [ ]:
!docker logs -f adminer

[Sat Apr 19 04:33:13 2025] PHP 8.4.6 Development Server (http://[::]:8080) started
[Sat Apr 19 04:33:27 2025] [::ffff:172.17.0.1]:41058 Accepted
[Sat Apr 19 04:33:27 2025] [::ffff:172.17.0.1]:41058 [200]: GET /
[Sat Apr 19 04:33:27 2025] [::ffff:172.17.0.1]:41058 Closing
[Sat Apr 19 04:33:27 2025] [::ffff:172.17.0.1]:41066 Accepted
[Sat Apr 19 04:33:27 2025] [::ffff:172.17.0.1]:41066 [200]: GET /?file=default.css&version=5.2.1
[Sat Apr 19 04:33:27 2025] [::ffff:172.17.0.1]:41066 Closing
[Sat Apr 19 04:33:27 2025] [::ffff:172.17.0.1]:41076 Accepted
[Sat Apr 19 04:33:27 2025] [::ffff:172.17.0.1]:41080 Accepted
[Sat Apr 19 04:33:27 2025] [::ffff:172.17.0.1]:41076 [200]: GET /?file=dark.css&version=5.2.1
[Sat Apr 19 04:33:27 2025] [::ffff:172.17.0.1]:41076 Closing
[Sat Apr 19 04:33:27 2025] [::ffff:172.17.0.1]:41080 [200]: GET /?file=functions.js&version=5.2.1
[Sat Apr 19 04:33:27 2025] [::ffff:172.17.0.1]:41080 Closing
[Sat Apr 19 04:33:27 2025] [::ffff:172.17.0.1]:41084 Accepted
[Sat Apr

In [ ]:
!docker inspect mysql | grep -n1 "IPAddress\":"


'grep' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
!docker inspect adminer| grep  -n1 "IPAddress\":"

'grep' is not recognized as an internal or external command,
operable program or batch file.


## Creating Named so that other containers can see other containers not just through the host

In [ ]:
#lets remove our old container
!docker stop adminer
!docker rm adminer
!docker stop mysql
!docker rm mysql

Error response from daemon: No such container: adminer
Error response from daemon: No such container: adminer
Error response from daemon: No such container: mysql
Error response from daemon: No such container: mysql


In [ ]:
# Create a network called yourNetwork using the following command

!docker network create yourNetwork

Error response from daemon: network with name yourNetwork already exists


In [ ]:
!docker network inspect yourNetwork

[
    {
        "Name": "yourNetwork",
        "Id": "2c48587edbe5da6904d38ddcad511bbbc6d0696d012b8dce23a209770c9ec02f",
        "Created": "2025-04-28T07:11:34.881620672Z",
        "Scope": "local",
        "Driver": "bridge",
        "EnableIPv4": true,
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": {},
            "Config": [
                {
                    "Subnet": "172.18.0.0/16",
                    "Gateway": "172.18.0.1"
                }
            ]
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {
            "ep-9c391aaf3c1d45f3e80919e225b329c21bb4919199e0f9549fb5edfc4c87286e": {
                "Name": "mysql",
                "EndpointID": "9c391aaf3c1d45f3e80919e225b329c21bb4919199e0f9549fb5edfc4c87286e",
                "MacAddress": "8e:da:66:2b:61:56"

In [ ]:
!docker run -d --network yourNetwork --name mysql -p 3306:3306 -e MYSQL_ROOT_PASSWORD=1 mysql:8


docker: Error response from daemon: Conflict. The container name "/mysql" is already in use by container "a2a1e44a8e92d1d8123858b288211f3b5a123055c2bc0616e10c0c5ba7d68bac". You have to remove (or rename) that container to be able to reuse that name.

Run 'docker run --help' for more information


In [ ]:
!docker run -d --network yourNetwork --name adminer -p 8080:8080 adminer

docker: Error response from daemon: Conflict. The container name "/adminer" is already in use by container "01078cc50140de2e5536aba8b10f438ab04eeba3637e5892a28d37c76603545f". You have to remove (or rename) that container to be able to reuse that name.

Run 'docker run --help' for more information


In [ ]:
#Now lets connect the adminer
!docker network connect yourNetwork adminer

Error response from daemon: endpoint with name adminer already exists in network yourNetwork


In [ ]:
!docker inspect mysql2 |grep -C5 " \"Networks\""

'grep' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
!docker inspect adminer| grep  -C5 " \"Networks\""

'grep' is not recognized as an internal or external command,
operable program or batch file.


# Part 2 
You will need to do a little digging here and work on your own!

Now that you have 2 containers up and running and know there connected. Create a python script that will check to see if the IP addresses of the containers are up and running. 


# Part 3
Continue with the above code and create a script that will relaunch a container if it ever shutsdown. please use the following web page to start your code and understand Docker API.

https://docs.docker.com/engine/api/sdk/examples/



In [ ]:
import docker
import subprocess

def get_container_ips(container_name):
    client = docker.from_env()
    ips = {}
    for name in container_name:
        container = client.containers.get(name)
        ip = container.attrs['NetworkSettings']['Networks']['yourNetwork']['IPAddress']
        ips[name] = ip
    return ips

def ping_container(ip):
    try:
        result = subprocess.run(['ping', '-c', '4', ip], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if result.returncode == 0:
            return True
        else:
            return False
    except Exception as e:
        print(f"Error pinging {ip}: {e}")
        return False
    
if __name__ == "__main__":
    container_names = ['mysql', 'adminer']
    ips = get_container_ips(container_names)
    
    for name, ip in ips.items():
        print(f"{name} IP: {ip}")
        if ping_container(ip):
            print(f"Ping to {name} ({ip}) successful.")
        else:
            print(f"Ping to {name} ({ip}) failed.")


mysql IP: 172.18.0.2
Ping to mysql (172.18.0.2) failed.
adminer IP: 172.18.0.3
Ping to adminer (172.18.0.3) failed.


In [ ]:
import docker
import subprocess  
import time

def get_container_ips(container_name):
    client = docker.from_env()
    ips = {}
    for name in container_name:
        container = client.containers.get(name)
        ip = container.attrs['NetworkSettings']['Networks']['yourNetwork']['IPAddress']
        ips[name] = ip
    return ips

def ping_container(ip):
    try:
        result = subprocess.run(['ping', '-c', '4', ip], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if result.returncode == 0:
            return True
        else:
            return False
    except Exception as e:
        print(f"Error pinging {ip}: {e}")
        return False
    
def ensure_container_running(container_name):
    client = docker.from_env()
    try:
        container = client.containers.get(container_name)
    except docker.errors.NotFound:
        print(f"Container {container_name} not found. Attempting to create it...")
        if container.status != 'running':
            print(f"Container {container_name} is not running. Restarting it...")
            try:
                container.restart()
                print(f"Container {container_name} restarted.")
            except Exception as e:
                print(f"Failed to restart container {container_name}: {e}")
        else:
            print(f"Container {container_name} is already running.")

if __name__ == "__main__":
    try:
        container_names = ['mysql', 'adminer']
    except Exception as e:
        print(f"An error occurred: {e}")

    while True:
        container_ips = get_container_ips(container_names)

        for name, ip in container_ips.items():
            is_up = ping_container(ip)
            if not is_up:
                print(f"Container {name} ({ip}) is down. Attempting to restart...")
                ensure_container_running(name)
            else:
                print(f"Container {name} ({ip}) is up and running.")
        time.sleep(10)  # Check every 10 seconds
# Note: This code is a simulation and will not run without a Docker daemon.


Container mysql (172.18.0.2) is down. Attempting to restart...
Container adminer (172.18.0.3) is down. Attempting to restart...
Container mysql (172.18.0.2) is down. Attempting to restart...
Container adminer (172.18.0.3) is down. Attempting to restart...


KeyboardInterrupt: 